In [ ]:
from bot.parse import *
from bot.state import *
from bot.model import *
from bot.ndlstm import separable_lstm
from bot.constants import MAX_BOARD_SIZE, FEATURE_SIZE, OUTPUT_SIZE


import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.contrib.rnn import GRUCell

In [ ]:
h = MAX_BOARD_SIZE
w = MAX_BOARD_SIZE
hidden_size = 7
learning_rate = 0.001
batch_size = 10
split = 0.2
train, test = train_test_split('training_data', 100, split)

# Build network
# Input placeholder
x = tf.placeholder(tf.float32, [None, h, w, FEATURE_SIZE])
# Output placeholder
y = tf.placeholder(tf.int32, [None, h, w])
mask = tf.placeholder(tf.int32, [None, 3])
seq_lengths = tf.placeholder(tf.int32, [None])

rnn_out = separable_lstm(GRUCell, hidden_size, x, seq_lengths)

model_out = slim.fully_connected(inputs=rnn_out,
                                 num_outputs=OUTPUT_SIZE,
                                 activation_fn=None)

predictions = tf.nn.top_k(model_out)
logits = tf.gather_nd(model_out, mask)
labels = tf.gather_nd(y, mask)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
loss = tf.reduce_mean(xentropy)
grad_update = tf.train.AdamOptimizer(learning_rate).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits, labels, 1), tf.float32))
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
split_1 = int(1/split)
samples = 5
with tf.Session() as sess:
    init.run()
    for i, training_file in enumerate(train):
        for j in range(samples):
            map_size, features, mask_idx, moves = data_gen(training_file, batch_size)
            sequence_lengths = [ map_size for i in range(64*batch_size)]
            feed_dict = {x: features, y:moves, mask: mask_idx, seq_lengths: sequence_lengths}
            sess.run([loss, grad_update], feed_dict=feed_dict)
            if not (i*j)%(split_1*samples):
                print('Training Accuracy: {}'.format(accuracy.eval(feed_dict=feed_dict)))
        if not i % split_1:
            test_file = test[int(i*0.2)]
            map_size, features, mask_idx, moves = data_gen(test_file, batch_size*15)
            sequence_lengths = [ map_size for i in range(64*batch_size*15)]
            feed_dict = {x: features, y:moves, mask: mask_idx, seq_lengths: sequence_lengths}
            print('Test Accuracy: {}'.format(accuracy.eval(feed_dict=feed_dict)))
            p = sess.run([predictions], feed_dict=feed_dict)
            print(p[0][1].shape)
            print(p[0][1])
    saver.save(sess, './model.ckpt')
            